In [3]:
import librosa
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
def extract_feature(file_name):
    X, sample_rate = librosa.core.load(file_name)
    result = np.array([])
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result = np.hstack((result, mel))
    return result

In [16]:
df = pd.read_table('targets.tsv')
df.head()

,Filename,Sex
0,5d1f7e43366513a1d0a6ec5640c3dc24,1
1,9a701a4536a05b6610a590a9fe702ed8,1
2,cad0b8547008d1524c1a0e5fd51f9908,1
3,4bbe607e7dc95460e2cc1a6ee5f4dfa6,0
4,30fb32cba90b34af26f3f14f5d636805,0


In [23]:
len(df)

13936

In [24]:
df['Sex'].value_counts()

0    7620
1    6316
Name: Sex, dtype: int64

In [25]:
for i, (filename,sex) in enumerate(zip(df['Filename'], df['Sex'])):
    pass

(13935, 'f8ef0441ebf79b817df790bb72d667c4', 0)

In [38]:
def load_train_data(vector_length=128):
    if not os.path.isdir("train_results"):
        os.mkdir("train_results")
    if os.path.isfile("train_results/features.npy") and os.path.isfile("train_results/labels.npy"):
        X = np.load("train_results/features.npy")
        y = np.load("train_results/labels.npy")
        return X, y
    n_samples = len(df)
    X = np.zeros((n_samples, vector_length))
    y = np.zeros((n_samples, 1))
    for i, (filename,sex) in tqdm(enumerate(zip(df['Filename'], df['Sex'])), "Loading data", total=n_samples):
        features = extract_feature('train/'+filename+'.wav')
        X[i] = features
        y[i] = sex
    np.save("train_results/features", X)
    np.save("train_results/labels", y)
    return X, y

In [44]:
X, y = load_train_data()

((13936, 128), (13936, 1))

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
def split_data(X, y, test_size=0.1, valid_size=0.1):
    # split training set and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=7)
    # split training set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }

In [60]:
X, y = load_train_data()
data = split_data(X, y, test_size=0.2, valid_size=0.1)

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [55]:
def init_model():
    model = Sequential()
    model.add(Dense(256, input_shape=(128,)))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    model.summary()
    return model

In [56]:
model = init_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0

In [57]:
tensorboard = TensorBoard(log_dir="logs")
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100

In [61]:
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

Epoch 1/100
157/157 [==============================] - 2s 6ms/step - loss: 0.4189 - accuracy: 0.8597 - val_loss: 0.2470 - val_accuracy: 0.9094
Epoch 2/100
157/157 [==============================] - 1s 5ms/step - loss: 0.2556 - accuracy: 0.9192 - val_loss: 0.1874 - val_accuracy: 0.9435
Epoch 3/100
157/157 [==============================] - 1s 4ms/step - loss: 0.2077 - accuracy: 0.9331 - val_loss: 0.1747 - val_accuracy: 0.9390
Epoch 4/100
157/157 [==============================] - 1s 4ms/step - loss: 0.1828 - accuracy: 0.9390 - val_loss: 0.1549 - val_accuracy: 0.9525
Epoch 5/100
157/157 [==============================] - 1s 4ms/step - loss: 0.1721 - accuracy: 0.9449 - val_loss: 0.1599 - val_accuracy: 0.9462
Epoch 6/100
157/157 [==============================] - 1s 5ms/step - loss: 0.1651 - accuracy: 0.9433 - val_loss: 0.1658 - val_accuracy: 0.9498
Epoch 7/100
157/157 [==============================] - 1s 4ms/step - loss: 0.1495 - accuracy: 0.9488 - val_loss: 0.1573 - val_accuracy: 0.9489

In [62]:
model.save("train_results/model.h5")

In [63]:
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Evaluating the model using 2788 samples...
Loss: 0.1091
Accuracy: 96.16%


In [76]:
features = extract_feature('train/00ceab975f6c7eceaa2b96d6e215b582.wav').reshape(1, -1)
female_prob = model.predict(features)[0][0]
male_prob = 1 - female_prob
gender = "male" if male_prob > female_prob else "female"
print("Result:", gender)
print(f"Probabilities::: Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")

Result: female
Probabilities::: Male: 0.00%    Female: 100.00%


In [1]:
from sklearn.model_selection import train_test_split
def split_data(X, y, test_size=0.1, valid_size=0.1):
    # split training set and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=7)
    # split training set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }

In [4]:
X = np.load("data/features.npy")
y = np.load("data/labels.npy")
data = split_data(X, y, test_size=0.2, valid_size=0.1)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [6]:
def init_model():
    model = Sequential()
    model.add(Dense(256, input_shape=(128,)))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    model.summary()
    return model
model = init_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               33024     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
tensorboard = TensorBoard(log_dir="logs")
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100

In [8]:
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

Epoch 1/100
754/754 [==============================] - 5s 5ms/step - loss: 0.5166 - accuracy: 0.7892 - val_loss: 0.3664 - val_accuracy: 0.8472
Epoch 2/100
754/754 [==============================] - 3s 4ms/step - loss: 0.3873 - accuracy: 0.8476 - val_loss: 0.3104 - val_accuracy: 0.8825
Epoch 3/100
754/754 [==============================] - 3s 4ms/step - loss: 0.3594 - accuracy: 0.8600 - val_loss: 0.2913 - val_accuracy: 0.8838
Epoch 4/100
754/754 [==============================] - 4s 5ms/step - loss: 0.3380 - accuracy: 0.8712 - val_loss: 0.2900 - val_accuracy: 0.8891
Epoch 5/100
754/754 [==============================] - 4s 5ms/step - loss: 0.3247 - accuracy: 0.8743 - val_loss: 0.2654 - val_accuracy: 0.8945
Epoch 6/100
754/754 [==============================] - 4s 6ms/step - loss: 0.3121 - accuracy: 0.8803 - val_loss: 0.2539 - val_accuracy: 0.9042
Epoch 7/100
754/754 [==============================] - 3s 5ms/step - loss: 0.3036 - accuracy: 0.8849 - val_loss: 0.2707 - val_accuracy: 0.8923

In [10]:
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Evaluating the model using 13388 samples...
Loss: 0.2178
Accuracy: 91.99%


In [11]:
model.save("data/model.h5")

In [46]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [47]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [52]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x))
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc4(x))
        return x

net = Net().float().to(device)
print(net)

Net(
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)
